# How to return an output file

## Questions

- Where does the output of a command go?

- How can I save the output of a command?

## Objectives

- Learn how to describe and handle outputs from a tool.

## Explanation 

The `outputs` of a tool is a list of output parameters that should be returned after running the tool. Each parameter has an `id` for the name of parameter, and `type` describing what types of values are valid for that parameter.

When a tool runs under CWL, the starting working directory is the designated output directory. The underlying tool or script must record its results in the form of files created in the output directory. The output parameters returned by the CWL tool are either the output files themselves, or come from examining the content of those files.

The following example demonstrates how to return a file generated by rasterio's `rio` command-line tool.

In [ ]:
cwltool --print-dot ../cwl/01-output/return-output-file.cwl 2> /dev/null | dot -Tpng | display

In [ ]:
ls -l ../cwl/01-output/return-output-file.cwl

In [ ]:
cwltool ../cwl/01-output/return-output-file.cwl \
    --tif "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/TCI.tif" > output.json

In [ ]:
cat output.json | yq e . - 

In [ ]:
cat ../cwl/01-output/return-output-file.cwl | yq e .'$graph[1].outputs' -

In [ ]:
# Extract inputs from the CWL file using yq
inputs=$(yq .'$graph[1].inputs' ../cwl/01-output/return-output-file.cwl)

# Format the extracted inputs as an HTML table
html="<h3>Workflow Inputs</h3><table border='1'><tr><th>Input Name</th><th>Type</th></tr>"
while read -r line; do
  if [[ "$line" =~ ^[[:space:]]*([^:]+): ]]; then
    input_name=${BASH_REMATCH[1]}
  elif [[ "$line" =~ ^[[:space:]]*type:[[:space:]]*(.*)$ ]]; then
    input_type=${BASH_REMATCH[1]}
    html+="<tr><td>${input_name}</td><td>${input_type}</td></tr>"
  fi
done <<< "$inputs"
html+="</table>"

# Render the HTML in the notebook
echo "$html" | displayHTML

In [ ]:
echo $inputs

In [ ]:
yq .'$graph[1].inputs' ../cwl/01-output/return-output-file.cwl